In [57]:
import geopandas as gpd
import requests
import pandas as pd

In [3]:
depts_mzns = gpd.read_file("inputs/departamentos_manzanas.geojson")

In [4]:
depts_mzns['DRIVE_ID'] = depts_mzns['LINK'].str.split('/', expand=True).iloc[:, 5]

In [5]:
depts_mzns['DRIVE_ID']

0      1xxixR0Bh8Oshq7uMCmVroOd4ZouL09-Q
1      1lnwxbXljpFvTZpMSPr8Vakx3BRNxBUms
2      1IZV20siTem9VlaAptAfoPLRWqV1XgLYV
3      1YA8djjIBKbnOi-YjBek5wBK7wdIF6ujT
4      1Dv-ZX-iaW0FNIAO5JIfM0fGgCWm0ymeQ
                     ...                
233    1BN-IktRh9pXw0G6E3iO4-GQ3VFiQ0uiK
234    1q4IDmZxKwaTvBwdtsRQZFbVtvdOG9jHp
235    1vXSUBqdi9UFdbsEJN89kEHDELaIO7VLy
236    1C0Z1Je1mvbqR77UEfhi_0eNh8VjQAf3G
237    101QW082rWQRg6X-k-4LedJpgNZK4bO6T
Name: DRIVE_ID, Length: 238, dtype: object

In [35]:
def download(file_id):
    URL = "https://drive.google.com/uc?iexport=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : file_id }, stream = True)

    token = get_confirm_token(response)

    if token:
        params = { 'id' : file_id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    gdf = gpd.read_file(response.raw)

    return gdf

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

In [33]:
resp = download('1xxixR0Bh8Oshq7uMCmVroOd4ZouL09-Q')

In [36]:
mzn_gdfs = []
descargados = 0
error = 0
for file_id in depts_mzns['DRIVE_ID']:
    try:
        gdf = download(file_id)
        mzn_gdfs.append(gdf)
        descargados += 1
    except:
        error += 1
        
    print(f'{descargados} Descargados / {error} Error', end='\r')

In [58]:
total_gdf = pd.concat(mzn_gdfs, ignore_index=True)

In [49]:
nombres = [gdf.iloc[0]['NOMBPROV'] for gdf in mzn_gdfs]

In [59]:
mzn_gdfs_error = []
descargados = 0
error = 0
for file_id in depts_mzns[~depts_mzns['NOMBPROV'].isin(nombres)]['DRIVE_ID']:
    try:
        gdf = download(file_id)
        mzn_gdfs_error.append(gdf)
        descargados += 1
    except:
        print(file_id)
        error += 1
        
    print(f'{descargados} Descargados / {error} Error', end='\r')

In [60]:
total_gdf_error = pd.concat(mzn_gdfs_error, ignore_index=True)

In [65]:
total_gdf_error['Mz'].isin(total_gdf['Mz']).any()

False

In [66]:
tota_gdf_final = pd.concat([total_gdf, total_gdf_error], ignore_index=True)

In [67]:
tota_gdf_final.to_file('outputs/manzanas_censales_peru.geojson', driver='GeoJSON')

/Users/claudio/.pyenv/versions/peru-connect-env/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
